## Coursera

In [1]:
import os
import time
import urllib.parse
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import json


In [4]:
options = webdriver.ChromeOptions()
options.add_argument("--lang=en-US")  # Force browser to load English
options.add_argument("--headless")  # Run in headless mode (no GUI)
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--log-level=3")  # Suppress logs

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# --------------------------------------------------------------------------------------------------------------------------
# Search skills: what skills do you want to get course data on
# --------------------------------------------------------------------------------------------------------------------------
# Specify the path to the file containing the extracted skills

with open("job_domain_skills.json", "r", encoding="utf-8") as f:
    job_domain_skills = json.load(f)

url_list = []

for job, skills in job_domain_skills.items():
    for skill in skills:
        url = "https://www.coursera.org/search?query="+urllib.parse.quote(f"{skill} {skill} {job}")
        url_list.append([job, skill, url])

latest_url_index = 0

data_folder = "coursera_data"
os.makedirs(data_folder, exist_ok=True)  # Ensure folder exists

# Function to scroll down to load dynamic content
def scroll_page(n,target_class):
    """scroll page utill there are n course cards or no new contents are loaded"""
    num_courses = len(driver.find_elements(By.CLASS_NAME, target_class))
    no_new_scroll = 0
    print(f"Scrolling and get num_course: {num_courses}",end="")
    while num_courses < n and no_new_scroll < 10:  # Adjust range as needed
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(0.5)# Wait for content to load
        new_num_courses = len(driver.find_elements(By.CLASS_NAME, target_class))
        print(f"-{new_num_courses}",end="")
        if new_num_courses > num_courses:
            no_new_scroll = 0
            num_courses = new_num_courses
        else:
            no_new_scroll += 1
    print('')

data = []
for idx, ls in enumerate(url_list[latest_url_index:]):
    job = ls[0]
    skill = ls[1]
    url = ls[2]
    print(job, skill)
    try:
        print(f"Scraping URL: {url}")
        driver.get(url)
        while not driver.page_source:
            time.sleep(3)# Wait for initial load
        
        # Define the class name of the target div (Update based on actual class)
        target_class = "cds-ProductCard-gridCard"  # Example class, adjust based on inspection
        
        # ------------------------------------------------------------------------------------------------------------------
        # Scroll page; change number of courses you want to scroll and scrape for each skill
        # ------------------------------------------------------------------------------------------------------------------
        scroll_page(20,target_class)

    except Exception as e:
        print(f"Failed to scrape {url}: {e}")

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all product containers
    course_divs = soup.find_all('div', class_= "cds-ProductCard-gridCard")
    print(f"Course divs found {len(course_divs)}")
    
    # Extract information
    course = []
    for div in course_divs:
        if div:
            # Extract course link
            link_div = div.find("a", class_="cds-119 cds-113 cds-115 cds-CommonCard-titleLink css-vflzcf cds-142")
            head = "https://www.coursera.org"
            link = link_div['href'] if link_div else None
            link = head + link if link else None

            # Extract course image link
            img_div = div.find("div", class_="cds-CommonCard-previewImage")
            img_tag = img_div.find("img") if img_div else None
            img_link = img_tag['src'] if img_tag else None

            # Extract partner name
            partner_tag = div.find('p', class_='cds-ProductCard-partnerNames css-vac8rf')
            partner = partner_tag.text.strip() if partner_tag else None

            # Extract course title
            title_tag = div.find('h3', class_='cds-CommonCard-title css-6ecy9b')
            title = title_tag.text.strip() if title_tag else None

            # Extract skills
            skill_div = div.find("div", class_="cds-ProductCard-body")
            skill_tag = skill_div.find('p', class_="css-vac8rf") if skill_div else None
            text = skill_tag.find_all(string=True, recursive=False) if skill_tag else None
            skills = [word.strip() for word in "".join(text).strip().split(",")] if text else None


            # Extract rating
            rating_tag = div.find('span', class_='css-6ecy9b')
            rating = float(rating_tag.text.strip()) if rating_tag else None

            def convert_reviews(text):
                text = text.replace("reviews", "").strip()  # Remove "reviews" and trim spaces
                if "K" in text:
                    return int(float(text.replace("K", "")) * 1000)  # Convert "2.1K" → 2100
                return int(text)  # Convert normal numbers

            # Extract number of reviews
            num_review_div = div.find("div", class_="cds-CommonCard-ratings")
            num_review_tag = num_review_div.find('div', class_='css-vac8rf') if num_review_div else None
            num_review = convert_reviews(num_review_tag.text.strip()) if num_review_tag else None

            # Extract course info
            info_div = div.find("div", class_="cds-CommonCard-metadata")
            info_tag = info_div.find('p', class_='css-vac8rf') if info_div else None
            info = info_tag.text.strip() if info_tag else None
            # Allowed difficulty levels
            difficulty_levels = {"Beginner", "Intermediate", "Advanced", "Mixed"}
            # Duration keywords
            duration_units = {"Weeks", "Months", "Hours"}
            def extract_info(text):
                parts = [part.strip() for part in text.split("·")]  # Split by '·' and clean spaces
                # Default buckets
                difficulty, course_type, duration = None, None, None
                for part in parts:
                    if part in difficulty_levels:
                        difficulty = part  # Assign as difficulty
                    elif any(unit in part for unit in duration_units):  # Check for duration keywords
                        duration = part
                    else:
                        course_type = part  # Assign as type if it's neither difficulty nor duration
                return difficulty, course_type, duration
            difficulty, course_type, duration = extract_info(info) if info else (None, None, None)
            
            # Append extracted data
            course.append({
                'skill': skill,
                'link': link,
                'image_link': img_link,
                'partner': partner,
                'title': title,
                'course_skill': skills,
                'rating': rating,
                'num_review': num_review,
                'difficulty': difficulty,
                'course_type': course_type,
                'duration': duration
            })
    df = pd.DataFrame(course)
    skillname = skill.replace("/", "_").replace("*", "_")
    csv_path = os.path.join(data_folder, f"{job}_{skillname}.csv")
    df.to_csv(csv_path, index=False)
    print(f"{skillname}.csv saved at: {csv_path}")
    print(f"-----------------------------------------------Scraping Complete Index ({latest_url_index+idx}/{len(url_list)-1})-----------------------------------------------")
            
# Close the driver after scraping
driver.quit()

Business Analyst .net
Scraping URL: https://www.coursera.org/search?query=.net%20.net%20Business%20Analyst
Scrolling and get num_course: 12-24
Course divs found 24
.net.csv saved at: coursera_data\Business Analyst_.net.csv
-----------------------------------------------Scraping Complete Index (0/2666)-----------------------------------------------
Business Analyst .net frameworks
Scraping URL: https://www.coursera.org/search?query=.net%20frameworks%20.net%20frameworks%20Business%20Analyst
Scrolling and get num_course: 12-24
Course divs found 24
.net frameworks.csv saved at: coursera_data\Business Analyst_.net frameworks.csv
-----------------------------------------------Scraping Complete Index (1/2666)-----------------------------------------------
Business Analyst a/b testing
Scraping URL: https://www.coursera.org/search?query=a/b%20testing%20a/b%20testing%20Business%20Analyst
Scrolling and get num_course: 12-24
Course divs found 24
a_b testing.csv saved at: coursera_data\Business Ana